<a href="https://colab.research.google.com/github/Thiraput01/QwenMed/blob/main/notebooks/QwenMed_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QwenMed: How to use

# Install libs

In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 10.3 MB/s eta 0:00:00


# Download the model using Unsloth (pulled from huggingface repository)

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM


tok_args = dict(trust_remote_code=True, token=False)
mdl_args = dict(torch_dtype="auto", device_map="auto", token=False)

tokenizer = AutoTokenizer.from_pretrained("Thiraput01/QwenMed-1.7B", **tok_args)
model     = AutoModelForCausalLM.from_pretrained("Thiraput01/QwenMed-1.7B", **mdl_args)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_atten

In [13]:
def inference(sys_prompt="You're a helpful assistant", prompt="Who are you?", enable_thinking=True):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    try:
    # rindex finding 151668 (</think>)
      index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    return thinking_content, content

In [14]:
thinking_content, content = inference(sys_prompt="You're a helpful assistant", prompt="Who are you?", enable_thinking=True)
print(thinking_content)
print(content)

<think>
Okay, let's see. I'm an AI assistant, right? I mean, that's what I'm supposed to be. I'm here to help with all sorts of things. Like, if you need help with homework, or if you're stuck on a coding problem, or even just want to chat about something interesting. I'm pretty versatile like that.

Hmm, I guess my main job is to support and assist people. I can answer questions, provide information, and help with tasks. And I'm always ready to help. So yeah, I'm definitely an AI assistant, and that's what I'm here for.
</think>
I'm an AI assistant, and I'm here to help with various tasks and provide support. Whether you need assistance with homework, coding, or just want to chat, I'm here to help. I'm always ready to support you in any way I can.


# Inference with **non thinking**

In [17]:
sys_prompt = """
You are a professional medical assistant powered by a dedicated reasoning model. Use your clinical reasoning skills to answer medical questions accurately and concisely. Follow these guidelines:
- Begin with a clear, patient-friendly answer.
- Highlight key points in brief bullet lists.
- Employ non-technical language whenever possible; explain unavoidable medical terms.
- Leverage your reasoning abilities to consider differential diagnoses and evidence-based options.
- If the user’s question is unclear, ask one precise clarifying question.
- Include a disclaimer that your advice is informational and not a replacement for an in-person evaluation by a qualified healthcare professional.
"""
prompt = "I'm having chest pain—what could be the cause?"

In [19]:
thinking_content, content = inference(sys_prompt=sys_prompt, prompt=prompt, enable_thinking=False)
print(thinking_content)
print(content)


Chest pain can be caused by a variety of conditions, and the specific cause often depends on the characteristics of the pain, the location, and any additional symptoms you are experiencing. Some common causes of chest pain include:

- Heart attack (myocardial infarction)
- Angina pectoris (angina)
- Costochondritis
- Pericarditis
- Aortic dissection
- Pneumothorax
- Myocarditis
- Esophagitis
- Esophageal spasm
- Aortic dissection
- Cardiovascular disease

If you are experiencing chest pain, it is important to seek medical attention immediately if you have any of the following:
- Chest pain that lasts more than 10 to 15 minutes
- Chest pain that worsens with deep breathing
- Sudden, severe chest pain
- A history of heart attack or other cardiac issues

Your healthcare provider will perform a physical examination and may order tests such as an electrocardiogram (ECG), chest X-ray, or blood tests to help determine the cause of your chest pain.


# Inference with **reasoning**

In [20]:
thinking_content, content = inference(sys_prompt=sys_prompt, prompt=prompt, enable_thinking=True)
print(thinking_content)
print(content)

<think>
Okay, let's think about what could be causing chest pain. Chest pain can be really uncomfortable and there are a lot of things that could cause it. First, there's heart-related stuff. Things like angina or even a heart attack could definitely cause chest pain. But, of course, it could also be something less serious like a muscle strain or maybe an infection in the chest, like pneumonia or even a lung problem.

Oh, and let's not forget about anxiety or stress. That can definitely lead to chest pain too. It's like the body's way of reacting to stress. So, if you're feeling really anxious or stressed, that's another thing to consider.

Now, if it's related to your heart, it's important to know the type of chest pain. Is it sharp or dull, or does it come and go? And what does it feel like? Also, how long has it been going on, and is it getting worse or better? These details can really help us figure out what's happening.

But wait, sometimes chest pain can be a sign of something mo

In [22]:
prompt = "I'm having an headace."
thinking_content, content = inference(sys_prompt=sys_prompt, prompt=prompt, enable_thinking=True)
print(thinking_content)
print(content)

<think>
Okay, so the user is experiencing a headache. Let's break this down. Headaches can be caused by a variety of things, like stress, overexertion, or even something as simple as not getting enough sleep. But when we're talking about a headache that's persistent or severe, it often points towards something more specific.

I should consider common causes first. Tension headaches and migraines are pretty typical, but they can also be part of a larger picture, like high blood pressure or something neurological. It's important to distinguish between acute and chronic headaches too.

The user might not have provided much detail, so it's crucial to ask them about the nature of the headache. Was it sudden or gradually getting worse? Any specific triggers or accompanying symptoms like nausea or vomiting could be pivotal clues.

Since headaches are often linked to lifestyle factors, I should ask about their daily routine. Are they stressed at work, or maybe overworked? Diet and sleep patter

# Test the performance with MedQA datasets

In [39]:
from datasets import load_dataset

ds = load_dataset("GBaker/MedQA-USMLE-4-options", token=False, split="test[:10%]")

In [40]:
ds

Dataset({
    features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
    num_rows: 127
})

In [41]:
ds['question'][0], ds['options'][0], ds['answer_idx'][0]

('A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?',
 {'A': 'Disclose the error to the patient and put it in the operative report',
  'B': 'Tell the attending that he cannot fail to disclose this mistake',
  'C': 'Report the physician to the ethics committee',
  'D': 'Refuse to dictate the operative report'},
 'B')

In [43]:
ds['options']

Column([{'A': 'Disclose the error to the patient and put it in the operative report', 'B': 'Tell the attending that he cannot fail to disclose this mistake', 'C': 'Report the physician to the ethics committee', 'D': 'Refuse to dictate the operative report'}, {'A': 'Inhibition of proteasome', 'B': 'Hyperstabilization of microtubules', 'C': 'Generation of free radicals', 'D': 'Cross-linking of DNA'}, {'A': 'Renal papillary necrosis', 'B': 'Cholesterol embolization', 'C': 'Eosinophilic granulomatosis with polyangiitis', 'D': 'Polyarteritis nodosa'}, {'A': 'Coagulase-positive, gram-positive cocci forming mauve-colored colonies on methicillin-containing agar', 'B': 'Encapsulated, gram-negative coccobacilli forming grey-colored colonies on charcoal blood agar', 'C': 'Spore-forming, gram-positive bacilli forming yellow colonies on casein agar', 'D': 'Lactose-fermenting, gram-negative rods forming pink colonies on MacConkey agar'}, {'A': 'Erythromycin ointment', 'B': 'Ketotifen eye drops', 'C'

In [46]:
import re
sys_prompt = (
    "You answer multiple-choice clinical ethics questions.\n"
    "Choose the single best option among A–D.\n"
    "Do NOT reveal your reasoning.\n"
    "Only output in the form: Final Answer: X\n"
)

def build_prompt(question: str, options: dict) -> str:
    # Ensure options are shown in A..D order
    letters = sorted(options.keys())
    opts_text = "\n".join(f"{L}. {options[L]}" for L in letters)
    return (
        f"Question:\n{question}\n\n"
        f"Options:\n{opts_text}\n\n"
        "Reply with: Final Answer: A/B/C/D"
    )

_choice_re = re.compile(r"\bFinal\s*Answer\s*[:\-]?\s*([ABCD])\b", re.IGNORECASE)

def extract_choice(text: str, options: dict) -> str | None:
    """
    Try to extract a single letter (A–D) from model output.
    Fall back to heuristic matches if needed.
    """
    if not text:
        return None

    # 1) Preferred: 'Final Answer: X'
    m = _choice_re.search(text)
    if m:
        return m.group(1).upper()

    # 2) Lone capital letter on a line
    for line in text.splitlines()[::-1]:  # check from bottom up
        line = line.strip()
        if line in {"A", "B", "C", "D"}:
            return line

    # 3) If the model echoed option text, map by longest matching option text
    candidates = []
    for L, opt_text in options.items():
        if opt_text and opt_text.strip() and opt_text.lower() in text.lower():
            candidates.append((len(opt_text), L))
    if candidates:
        # Prefer the longest matched option text
        candidates.sort(reverse=True)
        return candidates[0][1]

    # 4) As a last resort, pick the first seen letter token in text
    m2 = re.search(r"\b([ABCD])\b", text.upper())
    if m2:
        return m2.group(1)

    return None

def gold_letter(answer_idx, options: dict) -> str:
    """
    Normalize gold into a letter 'A'..'D'.
    Your dataset example shows 'answer_idx' as a letter ('B').
    If it's ever an int, we map by alphabetical order of keys.
    """
    if isinstance(answer_idx, str) and answer_idx in options:
        return answer_idx
    if isinstance(answer_idx, int):
        letters = sorted(options.keys())
        # Clamp just in case
        idx = max(0, min(answer_idx, len(letters) - 1))
        return letters[idx]
    # Fallback: try to coerce string like '1' -> index
    try:
        idx = int(answer_idx)
        letters = sorted(options.keys())
        idx = max(0, min(idx, len(letters) - 1))
        return letters[idx]
    except Exception:
        # If unknown format, raise to surface the issue early
        raise ValueError(f"Unrecognized answer_idx format: {answer_idx!r}")


In [55]:
from tqdm import tqdm
n = len(ds)
correct = 0
preds = []
fails = []

for i in tqdm(range(n), desc="Evaluating"):
    question = ds["question"][i]
    options  = ds["options"][i]
    gold     = gold_letter(ds["answer_idx"][i], options)

    prompt = build_prompt(question, options)
    thinking_content, content = inference(
        sys_prompt=sys_prompt,
        prompt=prompt,
        enable_thinking=True
    )

    pred = extract_choice(content, options)
    preds.append(pred)

    if pred == gold:
        correct += 1
    else:
        fails.append({
            "idx": i,
            "gold": gold,
            "pred": pred,
            "model_output": content
        })

acc = correct / n if n else 0.0
print(f"Accuracy: {acc:.3%} ({correct}/{n})")

Evaluating: 100%|██████████| 127/127 [41:04<00:00, 19.41s/it]

Accuracy: 37.008% (47/127)


In [56]:
for f in fails[:5]:
    print(f"\nItem {f['idx']} | gold={f['gold']} pred={f['pred']}\n{f['model_output']}")


Item 0 | gold=B pred=A
Final Answer: A

Item 1 | gold=D pred=A
Final Answer: A

Item 2 | gold=B pred=A
Final Answer: A

Item 3 | gold=D pred=A
Final Answer: A

Item 5 | gold=D pred=A
Final Answer: A. Diltiazem
